In [2]:
import os
import pandas as pd

In [3]:
df = pd.read_csv("chatbot-responses.csv")
df.head()

,conversation_id,question,response
0,1,Hello,"Hi, am Loki. How can I help you today?"
1,2,Hey,"Hi, am Loki. How can I help you today?"
2,2,Hi,"Hi, am Loki. How can I help you today?"
3,3,What's your name?,"My name is Loki, your helpful Kenyan travel as..."
4,4,Can you tell me about yourself?,"Of course! I'm Loki, a Kenyan bot specializing..."


In [5]:
from langchain_community.graphs import Neo4jGraph

In [6]:
graph = Neo4jGraph()

In [9]:
!neo4j config --database=neo4j set dbms.security.allow_csv_import_from_file_urls=true

Usage: neo4j { console | start | stop | restart | status | version }


In [ ]:
conversation_query = """
LOAD CSV WITH HEADERS FROM 'http://0.0.0.0:8000/chatbot-responses.csv' AS row
MERGE (m:Movie {id:row.movieId}
SET m.released = date(row.released
)
"""
graph.query(conversation_query);

In [ ]:
graph.refresh_schema(conversation_query)
print(graph.schema)